In [1]:
import pandas as pd
import re

folder = './052018_plate2/'

paths_read = [(folder + 'Results-Cyto-', 'Cytoplasm'),
              (folder + 'Results-Nuc-', 'Nucleus')]

paths_write = [folder + p[1] + '_May2018_2.xlsx' for p in paths_read]

markers = [('Nucleus (Cycle 1 - DAPI)', 1, 'DAPI'),
           ('BT474-H2BeGFP (Cycle 1 - FITC)', 1, 'FITC'),
           ('MCL1 (Cycle 2 - Cy3)', 2, 'Cy3'),
           ('p-p65NFkB (Cycle 2 - Cy5)', 2, 'Cy5'),
           ('p-Akt (Cycle 3 - FITC)', 3, 'FITC'),
           ('aSMA (Cycle 3 - Cy3)', 3, 'Cy3'),
           ('p-gH2AX (Cycle 3 - Cy5)', 3, 'Cy5'),
           ('GRP78 (Cycle 4 - FITC)', 4, 'FITC'),
           ('Vimentin (Cycle 4 - Cy3)', 4, 'Cy3'),
           ('Ki67 (Cycle 5 - FITC)', 5, 'FITC'),
           ('p-Rb (Cycle 5 - Cy3)', 5, 'Cy3'),
           ('FN (Cycle 5 - Cy5)', 5, 'Cy5'),
           ('FAP (Cycle 6 - Cy3)', 6, 'Cy3'),
           ('SOX2 (Cycle 6 - Cy5)', 6, 'Cy5')]

tabs = [('BT474,Cntrl(rb,c2-4)',   'B', [2,3,4]), 
        ('BT474,30nM(rb,c5-7)',    'B', [5,6,7]),
        ('BT474,300nM(rb,c8-10)',  'B', [8,9,10]),
        
        ('BT474+C3H-scr,Cntrl(rc,c2-4)',  'C', [2,3,4]),
        ('BT474+C3H-scr,30nM(rc,c5-7)',   'C', [5,6,7]),
        ('BT474+C3H-scr,300nM(rc,c8-10)', 'C', [8,9,10]),
        
        ('BT474+C3H-1,Cntrl(rd,c2-4)',  'D', [2,3,4]),
        ('BT474+C3H-1,30nM(rd,c5-7)',   'D', [5,6,7]),
        ('BT474+C3H-1,300nM(rd,c8-10)', 'D', [8,9,10]),
       
        ('BT+AR22,Cntrl(re,c2-4)',  'E', [2,3,4]),
        ('BT+AR22,30nM(re,c5-7)',   'E', [5,6,7]),
        ('BT+AR22,300nM(re,c8-10)', 'E', [8,9,10]),
        
        ('BT+hMSC,Cntrl(rf,c2-4)',  'F', [2,3,4]),
        ('BT+hMSC,30nM(rf,c5-7)',   'F', [5,6,7]),
        ('BT+hMSC,300nM(rf,c8-10)', 'F', [8,9,10])]


''' Construct the universal data file path in local drive named path
    Enter the row and a list of column indice you want to extract data 
    (e.g row E, column indice = [8, 9, 10])
    Construct a list of well IDs: B02-B10 named WELLS
    Construct a list of field IDs: fld1-fld9 named FIELD
'''

def read_df(path, rows, cols):
    WELLS = []
    
    for r in rows:
        for k in cols:
            if k < 10:
                WELLS.append(r + '0{}'.format(k))
            else:
                WELLS.append(r + '10')
    
    FIELD = ['fld{}'.format(k) for k in range(1, 10)]

    files = [path[0] + '{}{}.txt'.format(w, f) 
             for w in WELLS for f in FIELD]

    ''' Read all Cytoplams/Nucleus data files in the dataset into a list of dataframes
    '''
    
    df_list = [pd.read_csv(file, sep='\t', engine='python', 
                           usecols=['Label', 'IntDen']) 
                           for file in files]
    
    ''' Double check WELLS, FIELDs that have been processed in Cyto dataset
        Double check the number of files in the dataset that have been read
    '''
    print('\nRead', path[1], 'text files', '< ' * 3)
    print('\nWells:', WELLS)
    print('\nFields:', FIELD)
    print('\nNumber of files that have been read: ', len(df_list))    
    
    return df_list

def extract(df):
    well = r'[A-Z]+.*?(?=_)'
    field = r'(?<=d)[\d]{1,1}?(?=:)'
    cell_id = r'(\d+)-(\d+)'
    channel = r'(?<=:)[A-Z]+.*?(?=-)'
    
    df['Cycle'] = df['Label'].apply(lambda x: int(x[-1]))
    
    df['Channel'] = df['Label'].apply(lambda x: re.search(channel, x).group())
    
    df['Well'] = df['Label'].apply(lambda x: re.search(well, x).group())
    
    df['Field'] = df['Label'].apply(lambda x: re.search(field, x).group())
    
    df['Col'] = df['Well'].apply(lambda w: 10 if w[-1] == '0' else int(w[-1]))
    df['Group'] = df['Col'].apply(lambda c: (c - 2)//3)
    
    f = lambda x: re.search(cell_id, x).group().replace('-', '') 
    df['Cell ID'] = df['Label'].apply(f)
    
    g = lambda x: int(str(ord(x['Well'][0]) - 65) + str(x['Col'] - 1) + 
                      x['Field'] + x['Cell ID'])
    df['Cell ID'] = df.apply(g, axis=1)
    
    df = df[['Cell ID', 'Cycle', 'Channel', 'IntDen', 'Well', 'Field', 'Group']]
    
    return df

def process(df):
    dt = extract(df)

    df_field = dt[(dt['Cycle']==markers[0][1]) & 
                  (dt['Channel']==markers[0][2])][['Cell ID', 'Well', 'Field', 'Group']]

    for col in markers:
        dy = dt[(dt['Cycle']==col[1]) & (dt['Channel']==col[2])]
        df_field[col[0]] = dy[['IntDen']].values

    return df_field


In [2]:
for pr, pw  in zip(paths_read, paths_write):
    with pd.ExcelWriter(pw) as writer:
        print('\n', '$ ' * 12, 'Begin Processing', pr[1], 'Files', '$ ' * 12)
        for t in tabs:

            rows, cols = [t[1]], t[2]
            df = read_df(pr, rows, cols)

            df_cells = process(df[0])
            for df in df[1:]:
                dt = process(df)
                df_cells = df_cells.append(dt, ignore_index=True)
                
            print('\nWrite to', pw.replace(folder, ''), '> ' * 3)      
            df_cells.to_excel(writer, index=False, sheet_name=t[0])
            print('\n', '* ' * 40)


 $ $ $ $ $ $ $ $ $ $ $ $  Begin Processing Cytoplasm Files $ $ $ $ $ $ $ $ $ $ $ $ 

Read Cytoplasm text files < < < 

Wells: ['B02', 'B03', 'B04']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Cytoplasm_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Read Cytoplasm text files < < < 

Wells: ['B05', 'B06', 'B07']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Cytoplasm_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Read Cytoplasm text files < < < 

Wells: ['B08', 'B09', 'B10']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Cytoplasm_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * *


Read Nucleus text files < < < 

Wells: ['E08', 'E09', 'E10']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Nucleus_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Read Nucleus text files < < < 

Wells: ['F02', 'F03', 'F04']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Nucleus_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Read Nucleus text files < < < 

Wells: ['F05', 'F06', 'F07']

Fields: ['fld1', 'fld2', 'fld3', 'fld4', 'fld5', 'fld6', 'fld7', 'fld8', 'fld9']

Number of files that have been read:  27

Write to Nucleus_May2018_2.xlsx > > > 

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Read Nucleus text files < < < 

Wells: ['F08', 'F09', 'F10']

Fields: ['